# Viewing the Truth in a MicroBooNE LArTPC Event

This notebook provides examples of how to view important true information for the event.

Namely, for an event, we will view:
* 3D spacepoints representing the "locations" around the trajectory of simulated charged particles
* The true trajectories of particles generated by the simulation

Parts of our software we'll use:
* The interfaces to the different types of data objects we save (larlite and larcv): `larcv::IOManager` and `larlite::storage_manager`
* The class that creates and labels spacepoints from the 2D images: `larflow::PrepMatchTriplets`
* Convenience functions in our `lardly` repository for plotting the 2D and 3D data contained in the various objects



In [ ]:
# Import python modules and the plotly library
import os,sys # python modules
import plotly as pl # the entire plotly library
import plotly.graph_objects as go # provides functions to generate plot objects
import numpy as np # numpy library, provides definition of array data type used by plotly as input

# We include these jupyter magic lines
# This means that if we change any python code, we can reimport it with the updates.
%load_ext autoreload
%autoreload 2

In [ ]:
# Import the ROOT library
import ROOT as rt

In [ ]:
# import our ubdl libraries
from larlite import larlite # defines data products
from larcv import larcv # defines data products focused around dealing with images for deep learning
from ublarcvapp import ublarcvapp # provides functions that helps connect larlite and larcv data
from larflow import larflow # library for reconstruction functions

# classes and functions in the larutil namespace provides access to detector parameters
from larlite import larutil 

# our convenience functions that makes plotly plots from larlite or larcv data objects
import lardly

In [ ]:
# create instances of classes that are used to run various algorithms

# makes an image where pixels have value 1.0 if is due to bad wire channel, else 0.0 for good wire
badchmaker = ublarcvapp.EmptyChannelAlgo() 

You can get an example file from:

https://drive.google.com/drive/folders/1LOOLHZJYLmOZAiGT4IAgmZwgUGTRjBdD?usp=sharing

By default, the notebook will be configured to be able to process the following reference file:

`dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root`

This is simulated data for charge-current electron neutrino interactions for neutrinos whose energy distribution matches that of the Booster Neutrino Beam.
Simulated cosmic ray particles are also present.



In [ ]:
# Specify location of the file we want to open

inputfile_folder = "."
inputfile = "dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"


In [ ]:
# we'll want to plot the bounaries of the TPC
# and boundaries corresponding to the image when represented in 3D

import lardly
from lardly.detectoroutline import DetectorOutline
from lardly.evdimageoutline import EVDImageOutline

tpclines  = DetectorOutline().getlines(color=(0,0,0))
imgbounds = EVDImageOutline().getlines(color=(200,200,200))

# Run this block if you want to draw just the TPC plot that we retrieved
# To run, make the conditional `if True:`
if False:
    axis_template = {
        "showbackground": True,
        "backgroundcolor": "rgba(10,10,10,0.1)",
        "gridcolor": "rgb(10, 10, 10,0.2)",
        "zerolinecolor": "rgb(10,10,10,0.4)",
    }

    plot_layout = {
        "title": "",
        "height":800,
        "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
        "font": {"size": 12, "color": "black"},
        "showlegend": False,
        "plot_bgcolor": "white",
        "paper_bgcolor": "white",
        "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
        },
    }

    fig = go.Figure( data=tpclines+imgbounds, layout=plot_layout )
    fig.show()

We do not access the data (i.e. Trees) directly. The ROOT trees store data in the form of serialized instances of custom C++ classes! In otherwords, for each event, copies of C++ classes are turned into a binary string, storing the values of its data members. Unpacking this involves de-serializing the class data and giving back a set of C++ class instances for each event. We have special classes of our own that interfaces with ROOT's IO functions to help us do this.

There are two such interfacing classes we use to read the data.

The first is `larlite::storage_manager` which interfaces data products from larlite, which is a clone of the data products from larsoft. (Why do we use larlite and not larsoft directly? larlite does not require the large number of dependencies that are hard to build on systems that are not on Fermilab.)

The second is `larcv::IOManager` which provides the interface to the images, which we'll use at the end of the notebook.

In [ ]:
# We use the larcv IOManager class to load the trees for us.

# Long ago, we made the choice to put the origin of our images in the upper-left hand corner
# which was consistent with conventions used by image processing frameworks, in particular
# OpenCV. However, it is more natural for physicists to set the origin in the lower-right corner.
# But some old data still in use, including the official tutorial file, stores data 
# in the old upper-left hand "tick backward" convention.

# Change this to False if using newer data
LARCV_TICK_BACKWARD = True

# Initialize an storage_manager instance that will read out input file
ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
ioll.add_in_filename( inputfile_folder+"/"+inputfile )
ioll.set_verbosity(1)
ioll.open()

# Initialize an IOManager instance that will read out input file
if LARCV_TICK_BACKWARD:
    iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickBackward )
else:
    iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickForward )
    

iolcv.add_in_file( inputfile )
if LARCV_TICK_BACKWARD:
    iolcv.reverse_all_products()
iolcv.initialize()


In [ ]:
# Now we load an event

ENTRY_NUM = 0
ioll.go_to(ENTRY_NUM)
iolcv.read_entry(ENTRY_NUM)


In [ ]:
# Get the containers holding the mctrack and mcshower objects
# We print out info on the particles in the simulated event

event_mctrack  = ioll.get_data( larlite.data.kMCTrack,  "mcreco" )
event_mcshower = ioll.get_data( larlite.data.kMCShower, "mcreco" )

print("Number of mctracks: ",event_mctrack.size())
print("Number of mcshowers: ",event_mcshower.size())

# dump out data
print("============================")
print(" TRACKS ")
for itrack in range(event_mctrack.size()):
    track = event_mctrack.at(itrack)
    print("track[",itrack,"] pdg=",track.PdgCode()," E=",track.Start().E(),"pos=",track.Start().X(),track.Start().Y(),track.Start().Z())
    
print()    
print("============================")
print(" SHOWERS ")
for ishower in range(event_mcshower.size()):
    shower = event_mcshower.at(ishower)
    print("shower[",ishower,"] pdg=",shower.PdgCode()," E=",shower.Start().E()," pos=",shower.Start().X(),shower.Start().Y(),shower.Start().Z())
    print("  num daughter ids: ",shower.DaughterTrackID().size())


In [ ]:
# Before we vizualize this information, we can dump out this information
# We can use a utility that will try to arrange the list of tracks and showers
# by their mother-daughter relationship

# warning, its a bit of a mess

mcpg = ublarcvapp.mctools.MCPixelPGraph()
mcpg.buildgraphonly( ioll )
mcpg.printGraph(0, False)

Below we'll plot the Truth track trajectories. 

In the lardly code, the following color scheme is used:

```
default_pid_colors = {2212:'rgb(153,55,255)', # protons                                       
                      13:'rgb(255,0,0)', # muons                               
                      -13:'rgb(255,0,0)', # muons                          
                      211:'rgb(255,128,255)',# pions                       
                      -211:'rgb(255,128,255)',# pions                      
                      0:'rgb(0,0,0)'# other                                  
                      }
```

What you should notice is that all the tracks are inside the TPC.
This is because only true trajectory information is saved inside the TPC
(exception is the creation origin) which is kept somewhere in the mctrack object.

For showers, the lardly code plots them as a blue line with an associated cone.

The code that takes a container of mctrack objects and returns respective plot objects is

```
lardly.data.visualize_larlite_event_mctrack( event_mctrack, apply_t0_offset=False )
```

Note that `apply_t0_offset=False`.

In [ ]:
# make plot objects for the track objects
import lardly
apply_sce = False

plot_mctracks_nooffset  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=apply_sce,
                                                             no_offset=True )
print("mctrack plots: ",len(plot_mctracks))

plot_mcshowers_nooffset = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=True,
                                                              apply_sce=apply_sce,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
print("mcshower plots: ",len(plot_mcshowers))

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=tpclines+plot_mctracks_nooffset+plot_mcshowers_nooffset, layout=plot_layout )
fig.show()


Now we remake the track objects, but set `apply_t0_offset=False`.

What you'll see is that the tracks are now outside the TPC.
This has to do with the way we measure and reconstruct
the time of trajectories in the detector.


# DAQ and Timing in the TPC

To understand the timing in the detector, we need to describe how the data is saved.

There are electronics that continually stores a rolling queue of voltage versus time measurements.

The time between samples (sometimes called ticks) is determined by the electronics clock used. For MicroBooNE, the data acquisition system  (or DAQ) used to record the voltage versus time on all the wires saves a sample every 0.5 microseconds, i.e. 0.5 microseconds per tick.
There is some amount of stored values in the electronics. Values are replaced in a First-In-First-Out (FIFO) manner.

Every so often, a signal is sent to the electronics to save the stored values to disk.
This signal is called the trigger.
We use a number of different triggers. 
The primary one is a signal that comes a little bit ahead of when the neutrino beam will arrive.

The saved data is arranged in time order and a window of data is defined. 
This is how the data is arranged from MicroBooNE.
The tick that occured at the same time of the trigger is positioned to always be at `tick=3200`.
The range of samples stored goes from 3200 ticks before the trigger and 6400 ticks after the trigger to a total window size of 9600 ticks.
In order to save space, MicroBooNE in the first process of data processing removes the first 2400 ticks and 1152 ticks at the end for a truncated 6448 tick window.

In our code library, we retain this 2400-8448 tick range as the "coordinate" for the time axis of the image data.
In otherwords, we say the first "row" of the image is at tick=2400 and the last row ends at tick=8448.

Ionization left behind by charged particles in the detector arrive at the wireplanes and makes a signal in the electronics and thereby the image.  However, before the ionization arrives at the wireplane, the ionization must first drift from the location it was created to the wireplanes. Assuming a constant velocity (a first-order approximation we often use), the between the particle actually traveled the detector and the time the ionization shows up in the image is `(distance from wireplanes)/(drift velocity)`. The drift velocity depends on the electric field used in the detector. For MicroBooNE the drift velocity was measured to be around 0.111 cm per microseconds.

The original time of the trajectory (in others the real time) is known as the `t0` of the trajectory.
The `t0_offset` is the drift time or `(distance from wireplanes)/(drift velocity)`.  So the time that the signal appears on the wire plane is `t = t0 + t0_offset`.

So how can we find out this offset? Well, we actually are able to measure the original time of a trajectory.
In addition to ionization, most trajectories will produce a sizable flash of scintillation light that is seen in the optical detectors.  The time of the event can be measured potentially by the light is at the nanoseconds level.  However, associating flashes of light in the optical system to the charged trajectories is not an easy task and is not something we can do at the start.  You first have to decide what signals or pixels in an image are due trajectories occuring nearly at the same time.  Then you need to match that spatial pattern of ionization with the amount and spatial pattern of light signals in the optical detectors.  As a result, the `t0_offset` for a trajectory cannot be subtracted out until the middle or later portions of the reconstrucion.

Therefore, we often set the time of a trajectory as simply the relative to the original `tick=3200` event trigger.
And given that time offset, we use the const drift velocity to position the track in the detector. 

This is all to explain why when we plot the truth trajectories using the `apply_t0_offset` flag, the tracks are outside the TPC. This is because their timing (and equivalently) their position is all relative to `tick=3200`.
Plotting the true trajectories this way is useful for comparing against reconstruction trajectories before a `t0_offset` can be applied.


In [ ]:
# Plot the 3 Wire plane images

wire_tree_name = "wiremc"
# wire_tree_name = "wire" # use this if above does not work (usually for MC files)

ev_img = iolcv.get_data(larcv.kProductImage2D,wire_tree_name)
adc_v = ev_img.as_vector()

plane_plot = []
figs = []
for PLANE in range(adc_v.size()):
    plane_image = adc_v.at(PLANE)
    plane_plot.append(lardly.data.visualize_larcv_image2d( plane_image, reverse_ticks=False ))

    # plotly figure
    print("PLANE %d"%(PLANE))
    figs.append(  go.Figure( data=plane_plot ) )
    figs[-1].show()

## If opening the tutorial file, merged_dlreco_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root,
## the image should be of a Charged-Current Electron Neutrino Interaction
## If on the U-plane, the neutrino interaction vertex is around (1650,5328)

In [ ]:
# Now we take the input images and make spacepoint propopsals
from larflow import larflow
larflow.prep.PrepMatchTriplets
spacepoint_maker = larflow.prep.PrepMatchTriplets()

#  parameters
wire_image_tree_name = "wiremc"
badchannel_info_tree_name = "wiremc"
pixel_value_threshold = 10.0

# we pass the spacepoint making algorithm the entire larcv data interface.
# since we already loaded an event, it will use data from that event found in the input file
# to make spacepoints.
# the output information is stored inside data members in the class

# This function makes the 3D spacepoint proposals from the 2D images. 
# Note: this function is not fast, O(30 seconds per call)
print("Make spacepoints")
sys.stdout.flush()
spacepoint_maker.process( iolcv, wire_image_tree_name, badchannel_info_tree_name, pixel_value_threshold, True )

# Make the truth labels for the spacepoints:
# This includes: 
#   - true vs. ghost-tag
#   - the geant4 track id that deposited the most energy near this spacepoint. 
#         referred to as the "instance id" in the code
#   - the geant4 primary particle track ID that ultimately led to the current particle depositing energy. 
#         referred to as the "ancestor id" in the code
#   - the particle ID that deposited the most energy near this spacepoint
#         referred to as the "segment id" in the code
#   - origin label indiciting the source of the generator that the primary (or ancestor). labels:
#         0: unknown
#         1: neutrino
#         2: cosmic
print("Make truth labels")
sys.stdout.flush()
spacepoint_maker.process_truth_labels( iolcv, ioll, wire_image_tree_name )

In [ ]:
# once we have mode the spacepoint proposals, we can get various numpy arrays 
#  that provide various forms of information we get the spacepoints and their labels

# The function `make_triplet_ndarray` will return a dictionary with numpy arrays
# They contain:
#
# spacepoint_t: (x,y,z) position of the spacepoints.
#    The shape is (N,3)
#    The first dimension is the number of spacepoint proposals, typically O(100k)
#    The second dimension is the various labels
#    If we think of the array as carrying a 2D table,
#      the first dim are the rows representing data for individual spacepoints and
#      the second dim are the columns of different labels
#    The columns are:
#      [:,0] - the x-position of the spacepoints (with NO t0 drift time correction, nor space charge correction)
#      [:,1] - the y-position of the spacepoints (with no space charge correction)
#      [:,2] - the z-position of the spacepoints (with no space charge correction)
# 
# instance_t: the truth geant4 track id
#    The shape is (N,).
#    For ghost spacepoints the label is 0 
#      
# segment_t: the truth PID label of the spacepoint
#    Shape (N,)
#    For ghost spacepoints the label is 0
#
# origin_t: the origin flag for the spacepoints
#    Shape (N,)
#
# truetriplet_t: Label marking good and ghost spacepoints
#    Shape (N,)
#    0=ghost
#    1=true
#
# truespan_t: more of diagnostic quantity, 
#    this is how much deviation from a true 3-wire intersection was allowed in forming this proposal
# 

true_points_only = False
spacepoint_dict = spacepoint_maker.make_triplet_ndarray( true_points_only )
sys.stdout.flush()
print("Arrays returned: ")
for k in spacepoint_dict.keys():
    print(k,": ",spacepoint_dict[k].shape)

In [ ]:
# PLOT: True and ghost spacepoints

ONLY_TRUE_PTS = True

# get the numpy arrays we will use
pos_v = spacepoint_dict['spacepoint_t'] # 3D positions of spacepoints
ghost_label_v = spacepoint_dict['truetriplet_t'].astype(np.float) # labels for the spacepoints

# use numpy slice operations to select out the spacepoint rows that correspond to true energy deposit locations
if ONLY_TRUE_PTS:
    # numpy note: ghost_label_v[:]==1 returns an array of size (N,) with True and False 
    #  labels in each row. We use that flag to select a subsample of rows from pos_v and ghost_label_v arrays
    pos_v = pos_v[ ghost_label_v[:]==1,:]
    ghost_label_v = ghost_label_v[ ghost_label_v==1 ]
    print("check that the two arrays still have the same number of rows")
    print("nrows=",pos_v.shape[0]," matches=",pos_v.shape[0]==ghost_label_v.shape[0])

# we define a plotly 3D scatter plot
trace = {
    "type":"scatter3d",
    "x":pos_v[:,0],
    "y":pos_v[:,1],
    "z":pos_v[:,2],
    "mode":"markers",
    "name":"spacepoints",
    "marker":{"color":ghost_label_v,"size":1,"opacity":0.8,"colorscale":"Viridis"},
}

# some options to configure the way the plot looks that relate to the 3D axes
axis_template = { 
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

# more plot options, including initial camera settings
plot_layout = {
    "title":"SPACEPOINTS (ghost vs. true)",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

# collect all the things we want to be in our 3D plot into one list
data = tpclines+imgbounds+[trace]

fig = go.Figure( data=data, layout=plot_layout )
fig.show() 



# Running LArMAtch

The spacepoint proposals above, along with the input image, are passed to the larmatch network in order to generate several outputs:
* ghost vs. true spacepoint scores
* keypoint scores
* ssnet scores (though these typically underperform the 2D SSNet and are not used currently)

We plot the outputs contained within the output files of `deploy_larmatchme.py` found in `larflow/larmatchnet/larmatch`.

You can run the following two cells to run this program and generate an output file.

You'll need larmatch weights. The current official weight file is `checkpoint.cc1gpu.78000th.tar`.
You can find them on the Tufts cluster.

The usage instructions for `deploy_larmatchme.py`.

```
usage: run LArMatchMinkowskiEngine on data [-h] [--config-file CONFIG_FILE] 
    --supera SUPERA --weights WEIGHTS --output OUTPUT 
    [--tickbackwards] [--min-score MIN_SCORE] [--num-events NUM_EVENTS]
    [--has-mc] [--larlite-mcinfo LARLITE_MCINFO] [--has-wirecell] [--adc-name ADC_NAME]
    [--chstatus-name CHSTATUS_NAME] [--device-name DEVICE_NAME] [--use-skip-limit USE_SKIP_LIMIT]
```

Note: you need to have a working copy of PyTorch, MinkowskiEngine, and other libraries to be
able to run LArMatch.  It is recommended you run this is in a container on the Tufts Cluster.

In [ ]:
# set variables for running in larmatch in the cell on the input file

output_filename = "larmatchout_"+inputfile
checkpoint_file_folder=os.environ["UBDL_BASEDIR"]+"/larflow/larmatchnet/larmatch/"
checkpoint_file_name="checkpoint.cc1gpu.78000th.tar"
device_name="cpu"
#device_name="cuda:0"

x = os.path.abspath(inputfile_folder+"/"+inputfile)
y = os.path.abspath("./"+output_filename)

output_filename_larlite = "./"+output_filename.replace(".root","_larlite.root") 

os.environ["LM_input_filename"] = x
os.environ["LM_output_filename"] = y
os.environ["LM_checkpoint_folder"] = checkpoint_file_folder
os.environ["LM_checkpoint_filename"] = checkpoint_file_name
os.environ["LM_device_name"] = device_name


In [ ]:
%%bash

ORIG_FOLDER="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null 2>&1 && pwd )"
cd $UBDL_BASEDIR
cd larflow/larmatchnet
echo "Add ${PWD} to PYTHONPATH"
[[ ":$PYTHONPATH:" != *":${PWD}:"* ]] && export PYTHONPATH="${PWD}:${PYTHONPATH}"
#echo $PYTHONPATH
cd larmatch

ls -lh ${LM_input_filename}

python3 deploy_larmatchme.py --config config/config_larmatchme_deploycpu_tutorial.yaml \
    --supera ${LM_input_filename} --weights ${LM_checkpoint_folder}/${LM_checkpoint_filename} \
    --output ${LM_output_filename} --tickbackwards --min-score 0.0 --num-events 1 \
    --has-mc --larlite-mcinfo ${LM_input_filename} --adc-name=wiremc --chstatus-name=wiremc \
    --device-name ${LM_device_name} > tmp_notebook_larmatch_out

# You can watch the stdout as it is created by going into 
#   the larflow/larmatchnet/larmatch folder and running
#     tail -f tmp_notebook_larmatch_out

cd $ORIG_FOLDER
ls -lh ${LM_output_filename}

# Looking at LArMatch Output

If you have your own larmatch output files or you made your own, the output file comes in two types:
* a larcv file
* a larlite file

The larlite file contains the proposed spacepoints along the various scores produced by the network.


In [ ]:
# Open the larlite file containing larmatch spacepoints

# set the larmatch file name to this one if you made your own file
LM_OUTPUT_FILE = output_filename_larlite 

# set it yourself if you are using a pre-made file
LM_OUTPUT_FILE = ""

ioll_larmatch = larlite.storage_manager(larlite.storage_manager.kREAD)
ioll_larmatch.add_in_filename( LM_OUTPUT_FILE )
ioll_larmatch.set_verbosity(1)
ioll_larmatch.open()

# we instructed `deploy_larmatchme.py` to only run one event.
# so we go to entry 0
ioll_larmatch.go_to(0)
lmhits_v = ioll_larmatch.get_data( larlite.data.kLArFlow3DHit, "larmatch")
sys.stdout.flush()

print("Number of hits saved: ",lmhits_v.size())

The `larlite::larflow3dhit` class that stores info from the larmatch network was written for flexibility.
It simply inherits from `std::vector<float>`, which means there is just a simple list of numbers associated with each spacepoint.  

This is annoying because the class itself doesn't have any guiadance to what all the values in the vector mean.

The values in the vector are as follows:
* [0-2]:   x,y,z
* [3-8]:   deprecated
* [9]: the true (vs. ghost) score
* [10-16]: 7 ssnet scores, (bg,track,shower), from larmatch (not 2D sparse ssnet)
* [17-22]: 6 keypoint label score [nu,track-start,track-end,nu-shower,delta,michel]
* [23-25]: reserved for plane charge
* [26-28]: the predicted 3D direction (a unit vector) the particle is traveling in near this spacepoint

In [ ]:
# PLOT THE OUTPUT SCORES
import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=True

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="goodhit", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=9 )

plots = tpclines+imgbounds+[lfhit_plot]


if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=False,
                                                              return_detprofile=False)
    plots += plot_mctracks
    plots += plot_mcshowers
    print("mcshower plots: ",len(plot_mcshowers))
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()


In [ ]:
# PLOT THE KEYPOINT: NU SCORES

import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=False

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="goodhit", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=17 )

plots = tpclines+imgbounds+[lfhit_plot]

if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
    plots += plot_mctracks
    #plots += plot_mcshowers
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1.5, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()

In [ ]:
# PLOT THE KEYPOINT: TRACK-START

import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=False

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="track-start", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=18 )

plots = tpclines+imgbounds+[lfhit_plot]

if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
    plots += plot_mctracks
    #plots += plot_mcshowers
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1.5, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()

In [ ]:
# PLOT THE KEYPOINT: TRACK-END

import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=False

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="track-end[19]", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=19 )

plots = tpclines+imgbounds+[lfhit_plot]

if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
    plots += plot_mctracks
    #plots += plot_mcshowers
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1.5, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()

In [ ]:
# PLOT THE KEYPOINT: SHOWER-START
# These are primary showers

import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=False

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="shower[20]", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=20 )

plots = tpclines+imgbounds+[lfhit_plot]

if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
    plots += plot_mctracks
    #plots += plot_mcshowers
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1.5, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()

In [ ]:
# PLOT THE KEYPOINT: SHOWER-MICHEL
# These are e- or photons emitted by tracks, usually muons

import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=False

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="michel[21]", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=21 )

plots = tpclines+imgbounds+[lfhit_plot]

if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
    plots += plot_mctracks
    #plots += plot_mcshowers
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1.5, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()

In [ ]:
# PLOT THE KEYPOINT: SHOWER-DELTA
# These are e- or photons emitted by tracks, usually muons

import lardly

SCORE_THRESHOLD=0.5

# If you want to add truth trajectories to compare against the larmatch output
# You needed to run the cells above that lead to
# 
ADD_TRUTH_TRAJECTORIES=False

lfhit_plot = lardly.data.visualize_larlite_larflowhits( lmhits_v, name="delta[22]", 
                                                        score_threshold=SCORE_THRESHOLD,
                                                        max_hits=500000, score_index=22 )

plots = tpclines+imgbounds+[lfhit_plot]

if ADD_TRUTH_TRAJECTORIES:
    plot_mctracks  = lardly.data.visualize_larlite_event_mctrack( event_mctrack, 
                                                             do_sce_correction=False,
                                                             no_offset=False )
    plot_mcshowers = lardly.data.visualize_larlite_event_mcshower( event_mcshower,
                                                              no_offset=False,
                                                              apply_sce=False,
                                                              fixed_cone_len_cm=None,
                                                              return_origtraj_cone=True,
                                                              return_dirplot=True,
                                                              return_detprofile=False)
    plots += plot_mctracks
    #plots += plot_mcshowers
    

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1.5, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

fig = go.Figure( data=plots, layout=plot_layout )
fig.show()